# (N) Importing modules


In [ ]:
# connect Colab with Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# to import custom modules
import sys
sys.path.append("/content/drive/MyDrive/UNIMI/MachineLearning/UrbanSound/scripts")

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Activation , Dropout, InputLayer, Conv2D, MaxPooling2D, Flatten, Input, LSTM
from tensorflow.keras.layers.experimental import preprocessing
from keras import regularizers
from datetime import datetime
import os
import json

In [ ]:
import AddUtil
import DataLoader

In [ ]:
AddUtil.reset_random_seeds()

# (II) Basic model

In [ ]:
def cnn_model(X_train, 
              y_train,
              X_val, 
              y_val,
              X_test, 
              y_test, 
              ep=64, 
              bs=64, 
              name="cnn", 
              l_rate=0.001):
    
    # Ensure reproducibility
    AddUtil.reset_random_seeds()

    # Build the model
    model = Sequential()

    model.add(Conv2D(24, (5, 5), input_shape=(128, 128, 1)))
    model.add(MaxPooling2D((4, 2), (4, 2)))
    model.add(Activation('relu'))
          
    model.add(Conv2D(48, (5, 5)))
    model.add(MaxPooling2D((4, 2), (4, 2)))
    model.add(Activation('relu'))
          
    model.add(Conv2D(48, (5, 5)))
    model.add(Activation('relu'))

    model.add(Flatten())

    model.add(Dropout(0.5))
    model.add(Dense(64, kernel_regularizer=regularizers.l2(0.001)))
    model.add(Activation('relu'))

    model.add(Dropout(0.5))
    model.add(Dense(10, kernel_regularizer=regularizers.l2(0.001)))
    model.add(Activation('softmax'))

    # Compile the model
    opt = keras.optimizers.Adam(learning_rate=l_rate)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

    # Train the model
    history_model = model.fit(
        x=X_train, 
        y=y_train, 
        epochs=ep,
        batch_size=bs, 
        validation_data=(X_val, y_val),
        verbose=1)
    
    # Save accuracy and loss on the trainining and validation samples to the file
    history_dict_model = history_model.history
    AddUtil.save_metrics(history_dict_model,f"history_dict_{name}")

    # Save trained models (weights)
    model.save(f"/content/drive/MyDrive/UNIMI/MachineLearning/UrbanSound/pretrained_models/{name}")

    # Print training and validation loss and accuracy on the last epoch
    AddUtil.print_metrics(history_dict_model)

    # Evaluate the model on the test data:
    print(AddUtil.fold_evaluate(model, X_test, y_test))

# (III) Influence of the sample rate

## (3.1) CNN Model with sample_rate = 44100 Hz

### (3.1.1) Data Manipulation

In [ ]:
# Load features from the file
path1 = "/content/drive/MyDrive/UNIMI/MachineLearning/UrbanSound/features/spec_feat_with_stand_128_128_44100_1024_1024.npy"
df1 = DataLoader.create_df(path1)

In [ ]:
df1.info()

In [ ]:
df1.head(5)

In [ ]:
# Create training, validation and test subsets
X_train, y_train, X_val, y_val, X_test, y_test = DataLoader.train_val_test_split(df1)

In [ ]:
X_train.shape, X_val.shape, X_test[0].shape

In [ ]:
# cast size of training and validation sample to shape: (128, 128, 1)
X_train_ext = X_train[..., np.newaxis]
X_val_ext = X_val[..., np.newaxis]
print(X_train_ext.shape, X_val_ext.shape)

In [ ]:
# cast size of test sample to shape: (128, 128, 1)
X_test_ext = []
for item in X_test:
    X_test_ext.append(item[..., np.newaxis])

In [ ]:
for item in X_test_ext:
    print(item.shape)

### (3.1.2) Model

In [ ]:
EPOCHS = 64
BATCH_SIZE = 64
LEARNING_RATE = 0.001
NAME = "cnn1_44100"

In [ ]:
cnn1 = cnn_model(X_train_ext, 
                 y_train, 
                 X_val_ext, 
                 y_val, 
                 X_test_ext, 
                 y_test,
                 ep=EPOCHS,
                 bs=BATCH_SIZE,
                 name=NAME,
                 l_rate=LEARNING_RATE)

## (3.2) CNN Model with sample_rate = 22050 Hz

### (3.2.1) Data Manipulation

In [ ]:
%reset

In [ ]:
# Load features from the file
path2 = "/content/drive/MyDrive/UNIMI/MachineLearning/UrbanSound/features/spec_feat_with_stand_128_128_22050_1024_1024.npy"
df2 = DataLoader.create_df(path2)

In [ ]:
df2.info()

In [ ]:
# Create training, validation and test subsets
X_train, y_train, X_val, y_val, X_test, y_test = DataLoader.train_val_test_split(df2)

In [ ]:
# cast size of training and validation sample to shape: (128, 128, 1)
X_train_ext = X_train[..., np.newaxis]
X_val_ext = X_val[..., np.newaxis]
print(X_train_ext.shape, X_val_ext.shape)

In [ ]:
# cast size of test sample to shape: (128, 128, 1)
X_test_ext = []
for item in X_test:
    X_test_ext.append(item[..., np.newaxis])

In [ ]:
for item in X_test_ext:
    print(item.shape)

### (3.2.2) Model

In [ ]:
EPOCHS = 64
BATCH_SIZE = 64
LEARNING_RATE = 0.001
NAME = "cnn1_22050"

In [ ]:
cnn1 = cnn_model(X_train_ext, 
                 y_train, 
                 X_val_ext, 
                 y_val, 
                 X_test_ext, 
                 y_test,
                 ep=EPOCHS,
                 bs=BATCH_SIZE,
                 name=NAME,
                 l_rate=LEARNING_RATE)

## (3.3) CNN Model with sample_rate = 11025 Hz

### (3.3.1) Data Manipulation

In [ ]:
# Load features from the file
path3 = "/content/drive/MyDrive/UNIMI/MachineLearning/UrbanSound/features/spec_feat_with_stand_128_128_11025_1024_1024.npy"
df3 = DataLoader.create_df(path3)
df3.info()

In [ ]:
# Create training, validation and test subsets
X_train, y_train, X_val, y_val, X_test, y_test = DataLoader.train_val_test_split(df3)

In [ ]:
# cast size of training and validation sample to shape: (128, 128, 1)
X_train_ext = X_train[..., np.newaxis]
X_val_ext = X_val[..., np.newaxis]
print(X_train_ext.shape, X_val_ext.shape)

In [ ]:
# cast size of test sample to shape: (128, 128, 1)
X_test_ext = []
for item in X_test:
    X_test_ext.append(item[..., np.newaxis])

for item in X_test_ext:
    print(item.shape)

### (3.3.2) Model

In [ ]:
EPOCHS = 64
BATCH_SIZE = 64
LEARNING_RATE = 0.001
NAME = "cnn1_11025"

In [ ]:
cnn3 = cnn_model(X_train_ext, 
                 y_train, 
                 X_val_ext, 
                 y_val, 
                 X_test_ext, 
                 y_test,
                 ep=EPOCHS,
                 bs=BATCH_SIZE,
                 name=NAME,
                 l_rate=LEARNING_RATE)

## (3.4) CNN Model with sample_rate = 5512.5 Hz

In [ ]:
%reset

### (3.4.1) Data Manipulation

In [ ]:
# Load features from the file
path4 = "/content/drive/MyDrive/UNIMI/MachineLearning/UrbanSound/features/spec_feat_with_stand_128_128_5512.5_1024_1024.npy"
df4 = DataLoader.create_df(path4)
df4.info()

In [ ]:
# Create training, validation and test subsets
X_train, y_train, X_val, y_val, X_test, y_test = DataLoader.train_val_test_split(df4)

In [ ]:
# cast size of training and validation sample to shape: (128, 128, 1)
X_train_ext = X_train[..., np.newaxis]
X_val_ext = X_val[..., np.newaxis]
print(X_train_ext.shape, X_val_ext.shape)

In [ ]:
# cast size of test sample to shape: (128, 128, 1)
X_test_ext = []
for item in X_test:
    X_test_ext.append(item[..., np.newaxis])

for item in X_test_ext:
    print(item.shape)

### (3.4.2) Model

In [ ]:
EPOCHS = 64
BATCH_SIZE = 64
LEARNING_RATE = 0.001
NAME = "cnn1_5512_5"

In [ ]:
cnn4 = cnn_model(X_train_ext, 
                 y_train, 
                 X_val_ext, 
                 y_val, 
                 X_test_ext, 
                 y_test,
                 ep=EPOCHS,
                 bs=BATCH_SIZE,
                 name=NAME,
                 l_rate=LEARNING_RATE)

# (IV) Data Manipulation

Extracting feature for the sample_rate = 11025 Hz for the further experiments.

In [ ]:
# Load features from the file
path = "/content/drive/MyDrive/UNIMI/MachineLearning/UrbanSound/features/spec_feat_with_stand_128_128_11025_1024_1024.npy"
df = DataLoader.create_df(path)
df.info()

In [ ]:
# Create training, validation and test subsets
X_train, y_train, X_val, y_val, X_test, y_test = DataLoader.train_val_test_split(df)

In [ ]:
# cast size of training and validation sample to shape: (128, 128, 1)
X_train_ext = X_train[..., np.newaxis]
X_val_ext = X_val[..., np.newaxis]
print(X_train_ext.shape, X_val_ext.shape)

In [ ]:
# cast size of test sample to shape: (128, 128, 1)
X_test_ext = []
for item in X_test:
    X_test_ext.append(item[..., np.newaxis])

for item in X_test_ext:
    print(item.shape)

# (V) Influence of learning rate

## l_rate = 0.00001

In [ ]:
%reset

In [ ]:
EPOCHS = 64
BATCH_SIZE = 64
LEARNING_RATE = 0.00001
NAME = "cnn_lr_00001"

In [ ]:
print(EPOCHS, BATCH_SIZE, LEARNING_RATE, NAME)

In [ ]:
cnn_model(X_train_ext, 
              y_train, 
              X_val_ext, 
              y_val, 
              X_test_ext, 
              y_test,
              ep=EPOCHS,
              bs=BATCH_SIZE,
              name=NAME,
              l_rate=LEARNING_RATE)

## l_rate = 0.0001

In [ ]:
%reset

In [ ]:
EPOCHS = 64
BATCH_SIZE = 64
LEARNING_RATE = 0.0001
NAME = "cnn_lr_0001"

In [ ]:
print(EPOCHS, BATCH_SIZE, LEARNING_RATE, NAME)

In [ ]:
cnn_model(X_train_ext, 
              y_train, 
              X_val_ext, 
              y_val, 
              X_test_ext, 
              y_test,
              ep=EPOCHS,
              bs=BATCH_SIZE,
              name=NAME,
              l_rate=LEARNING_RATE)

## l_rate = 0.001

In [ ]:
%reset

In [ ]:
EPOCHS = 64
BATCH_SIZE = 64
LEARNING_RATE = 0.001
NAME = "cnn_lr_001"

In [ ]:
print(EPOCHS, BATCH_SIZE, LEARNING_RATE, NAME)

In [ ]:
cnn_model(X_train_ext, 
              y_train, 
              X_val_ext, 
              y_val, 
              X_test_ext, 
              y_test,
              ep=EPOCHS,
              bs=BATCH_SIZE,
              name=NAME,
              l_rate=LEARNING_RATE)

## l_rate = 0.01

In [ ]:
%reset

In [ ]:
EPOCHS = 64
BATCH_SIZE = 64
LEARNING_RATE = 0.01
NAME = "cnn_lr_01"

In [ ]:
print(EPOCHS, BATCH_SIZE, LEARNING_RATE, NAME)

In [ ]:
cnn_model(X_train_ext, 
              y_train, 
              X_val_ext, 
              y_val, 
              X_test_ext, 
              y_test,
              ep=EPOCHS,
              bs=BATCH_SIZE,
              name=NAME,
              l_rate=LEARNING_RATE)

# (VI) Influence of batch size

## batch size = 32

In [ ]:
EPOCHS = 64
BATCH_SIZE = 32
LEARNING_RATE = 0.0001
NAME = "cnn_bs_32"

In [ ]:
print(EPOCHS, BATCH_SIZE, LEARNING_RATE, NAME)

In [ ]:
cnn_model(X_train_ext, 
              y_train, 
              X_val_ext, 
              y_val, 
              X_test_ext, 
              y_test,
              ep=EPOCHS,
              bs=BATCH_SIZE,
              name=NAME,
              l_rate=LEARNING_RATE)

## batch size = 128

In [ ]:
EPOCHS = 64
BATCH_SIZE = 128
LEARNING_RATE = 0.0001
NAME = "cnn_bs_128"

In [ ]:
print(EPOCHS, BATCH_SIZE, LEARNING_RATE, NAME)

In [ ]:
cnn_model(X_train_ext, 
              y_train, 
              X_val_ext, 
              y_val, 
              X_test_ext, 
              y_test,
              ep=EPOCHS,
              bs=BATCH_SIZE,
              name=NAME,
              l_rate=LEARNING_RATE)

## batch size = 16

In [ ]:
EPOCHS = 64
BATCH_SIZE = 16
LEARNING_RATE = 0.0001
NAME = "cnn_bs_16"

In [ ]:
print(EPOCHS, BATCH_SIZE, LEARNING_RATE, NAME)

In [ ]:
cnn_model(X_train_ext, 
              y_train, 
              X_val_ext, 
              y_val, 
              X_test_ext, 
              y_test,
              ep=EPOCHS,
              bs=BATCH_SIZE,
              name=NAME,
              l_rate=LEARNING_RATE)

## batch size = 8

In [ ]:
EPOCHS = 64
BATCH_SIZE = 8
LEARNING_RATE = 0.0001
NAME = "cnn_bs_8"

In [ ]:
print(EPOCHS, BATCH_SIZE, LEARNING_RATE, NAME)

In [ ]:
cnn_model(X_train_ext, 
              y_train, 
              X_val_ext, 
              y_val, 
              X_test_ext, 
              y_test,
              ep=EPOCHS,
              bs=BATCH_SIZE,
              name=NAME,
              l_rate=LEARNING_RATE)